In [1]:
import sys
!{sys.executable} -m pip install redis

You should consider upgrading via the '/Users/Jeka/anaconda3/bin/python -m pip install --upgrade pip' command.


In [2]:
!{sys.executable} -m pip install bson

You should consider upgrading via the '/Users/Jeka/anaconda3/bin/python -m pip install --upgrade pip' command.


In [63]:
import redis 

In [64]:
r = redis.Redis(host='localhost', port=6379, db=0)

In [65]:
from ipywidgets import HTML

def show_redis_data(r):
    html = HTML()
    out = '<table border="1">'
    for key in r.keys():
        t = r.type(key).decode()
#         print(key.decode(), t)
        
        if t == 'string':
            data = r.get(key)
            out += f'<tr><td style="padding:3px;width:150px;">{key.decode()}</td>'\
                   f'<td style="padding:3px;width:50px;">string</td>'\
                   f'<td style="padding:3px;">{data.decode()}</td></tr>'
        elif t == 'list':
            l = r.llen(key)
            data = r.lrange(key, 0, l)
            out += f'<tr><td style="padding:3px;width:150px;">{key.decode()}</td>'\
                   f'<td style="padding:3px;width:50px;">list</td>'\
                   f'<td style="padding:3px;">[{", ".join([d.decode() for d in data])}]</td></tr>'
        elif t == 'hash':
            data = []
            for key2 in r.hkeys(key):
                data.append(f'{key2.decode()}: {r.hget(key, key2).decode()}')
            out += f'<tr><td style="padding:3px;width:150px;">{key.decode()}</td>'\
                   f'<td style="padding:3px;width:50px;">hash</td>'\
                   f'<td style="padding:3px;">{{{", ".join(data)}}}</td></tr>'
        elif t == 'set':
            data = r.smembers(key)
            out += f'<tr><td style="padding:3px;width:150px;">{key.decode()}</td>'\
                   f'<td style="padding:3px;width:50px;">set</td>'\
                   f'<td style="padding:3px;">({", ".join([d.decode() for d in data])})</td></tr>'
        
    out += '</table>'
    
    html.value = out
    display(html)

In [66]:
zhenya = {
            'person_id': 0,
            'name': 'Женя',
            'gender': 'ж',
            'age': '20',
            'country': 'RF',
            'distance': 0
        }

nastya = {
            'person_id': 1,
            'name': 'Настя Чижикова',
            'gender': 'ж',
            'age': '20',
            'country': 'RF',
            'distance': 0
        }

ritos = {
            'person_id': 2,
            'name': 'Рита Попова',
            'gender': 'ж',
            'age': '23',
            'country': 'RF',
            'distance': 0
        }

lilya = {
            'person_id': 3,
            'name': 'Лиля',
            'gender': 'ж',
            'age': '22',
            'country': 'Arzamas',
            'distance': 0
        }


kuzya = {
            'person_id': 4,
            'name': 'Кузя',
            'gender': 'ж',
            'age': '22',
            'country': 'Чехия',
            'distance': 0
        }

polyana = {
            'person_id': 5,
            'name': 'Поляна',
            'gender': 'ж',
            'age': '21',
            'country': 'RF',
            'distance': 0
        }

danya = {
            'person_id': 6,
            'name': 'Даня',
            'gender': 'м',
            'age': '25',
            'country': 'Грузия',
            'distance': 0
        }

participants_list = [zhenya, nastya, ritos, lilya, kuzya, polyana, danya]

In [67]:
import random
from bson import ObjectId

In [68]:
r = redis.Redis(host='localhost', port=6379, db=0)

In [69]:
r.flushdb()

True

In [70]:
def AddNewRunner(cur_time, person_info=None):
    if person_info == None:
        oid = str(ObjectId())
        age = random.randint(18, 70)
        gender = random.choice(['m', 'f'])
        country = random.choice(['USA', 'CANADA', 'FRANCE', 'UKRAINE'])
        name = random.choice(['Mike', 'Michael', 'Michelle', 'Misha'])
        start_time = cur_time    
        distance = 0

        person_info = {
            'person_id': oid,
            'name': name,
            'gender': gender,
            'age':age,
            'country': country,
            'distance': distance
        }
        
    # новый человек начинает бежать, надо добавить про него инфу (CREATE)
    person_info['start_time'] = str(cur_time) #он только начал
    r.hset("person:"+str(person_info["person_id"]), mapping=person_info) 
    # и добавить его в список атлетов
    r.lpush("person_ids", person_info["person_id"])
    
    return person_info['person_id']
    

def AddNewRecord(oid, cur_time, record_info=None):
    timestamp = cur_time
    if record_info == None:
        if cur_time != 0: 
            #усложняю себе жизнь полуадекватной генерацией
            if r.exists(str(oid)+':'+str(cur_time - 1)) == 1: #если этот человек уже есть и бежит

                passed_distance = r.hget(str(oid)+':'+str(cur_time - 1), 'distance')
                distance = int(passed_distance.decode()) + random.randint(1, 7)

                prev_pulse_rate = r.hget(str(oid)+':'+str(cur_time - 1), 'pulse_rate')
                prev_pulse_rate = int(prev_pulse_rate.decode())
                if prev_pulse_rate < 180:
                    pulse_rate = random.randint(prev_pulse_rate - 5, prev_pulse_rate + 10)
                else:
                    pulse_rate = random.randint(prev_pulse_rate - 10, prev_pulse_rate)
                
            else:  #если этого человека нет и он начал бежать
                distance = 0 
                pulse_rate = random.randint(60, 90)
                r.hset("person:"+str(oid), 'start_time', cur_time)
        
        else:  #если самое начало отчета для всех
            distance = 0
            pulse_rate = random.randint(60, 90)
            r.hset("person:"+str(oid), 'start_time', cur_time)
            
        record_info = {
        'distance': distance,
        'pulse_rate': pulse_rate
        }
       
    
    r.hset(str(oid)+':'+str(cur_time), mapping=record_info)
    r.hset("person:"+str(oid), 'distance', record_info['distance'])  #UPDATE
        
            
def StartRunner(cur_time=0, person_info=None):
    #создадим человека или заменим старт на текущее время
    if person_info != None: 
        oid = person_info['person_id']
        if r.exists('person:'+str(oid)) == 1:  # человек уже есть в базе, но решил начать заново (UPDATE)
            r.hset('person:'+str(oid), 'start_time', cur_time)
            r.hset('person:'+str(oid), 'distance', 0)
        else:
            oid = AddNewRunner(cur_time, person_info) #человека в базе нет, создаем его
    else:
        oid = AddNewRunner(cur_time)
    
    #создадим запись о том, что они начали
    AddNewRecord(oid, cur_time)
    
    return oid
                     
         

In [73]:
StartRunner()
show_redis_data(r)

HTML(value='<table border="1"><tr><td style="padding:3px;width:150px;">0:1</td><td style="padding:3px;width:50…

In [74]:
AddNewRecord('0', 2)
show_redis_data(r)

HTML(value='<table border="1"><tr><td style="padding:3px;width:150px;">0:1</td><td style="padding:3px;width:50…

In [75]:
r.flushdb()

True

In [76]:
def CreateRace(participants_list, time_sec):
    for participant in participants_list:
        StartRunner(0, participant)
        

    person_ids = [oid.decode() for oid in r.lrange('person_ids', 0, -1)]
    
    for i in range(time_sec): #секунды
        r.set('cur_time', i)  
                
        for oid in person_ids:
            AddNewRecord(oid, i)
            
        if i == 10:
            for num in range(5):  #еще 5 рандомных через 100 секунд
                oid = StartRunner(cur_time=i)
            person_ids = [oid.decode() for oid in r.lrange('person_ids', 0, -1)]
    
    

In [77]:
CreateRace(participants_list, 100)
show_redis_data(r)

HTML(value='<table border="1"><tr><td style="padding:3px;width:150px;">4:23</td><td style="padding:3px;width:5…

# READ

In [78]:
# посмотреть, кто на какой дистанции находится в определенный момент времени 
def check_at_time(time_check):
    person_ids = [oid.decode() for oid in r.lrange('person_ids', 0, -1)]
    names = []
    if time_check > int(r.get('cur_time').decode()):
            print('это время еще не настало')
    else:
        for participant in person_ids: 
            if r.exists(str(participant)+':'+str(time_check)):
                k = r.hget(str(participant)+':'+str(time_check), 'distance') #+st
                for rp in participants_list:
                    if str(rp['person_id']) == participant:
                        participant = rp['name']
                print(participant, k.decode())
            

In [79]:
check_at_time(2)

Даня 10
Поляна 12
Кузя 8
Лиля 11
Рита Попова 13
Настя Чижикова 3
Женя 9


In [80]:
check_at_time(99)

62444888524f204a15ee4a3a 342
62444888524f204a15ee4a39 346
62444888524f204a15ee4a38 348
62444888524f204a15ee4a37 358
62444888524f204a15ee4a36 362
Даня 390
Поляна 387
Кузя 437
Лиля 401
Рита Попова 402
Настя Чижикова 393
Женя 360


# SORT

отсортировать всех людей в базе по возрасту

In [81]:
def sort_by_age(): #сформировать множество и отсортировать
    person_ids = r.lrange('person_ids', 0, -1)
    
    for line in ['person:'+person_id.decode() for person_id in person_ids]:
        r.sadd('indices', line) #добавить всех person
        
    a = r.smembers('indices') 
    print(a)
    
    sorted_pers = r.sort_ro('indices', by='*->age')
    print(sorted_pers)
    
    for person_id in sorted_pers: 
        print(r.hget(person_id.decode(), 'name').decode(), r.hget(person_id.decode(), 'age').decode())
        

In [82]:
sort_by_age()

{b'person:62444888524f204a15ee4a3a', b'person:4', b'person:1', b'person:5', b'person:62444888524f204a15ee4a36', b'person:6', b'person:62444888524f204a15ee4a39', b'person:2', b'person:62444888524f204a15ee4a38', b'person:3', b'person:62444888524f204a15ee4a37', b'person:0'}
[b'person:62444888524f204a15ee4a38', b'person:0', b'person:1', b'person:5', b'person:3', b'person:4', b'person:2', b'person:6', b'person:62444888524f204a15ee4a37', b'person:62444888524f204a15ee4a39', b'person:62444888524f204a15ee4a36', b'person:62444888524f204a15ee4a3a']
Michelle 19
Женя 20
Настя Чижикова 20
Поляна 21
Лиля 22
Кузя 22
Рита Попова 23
Даня 25
Mike 28
Misha 33
Michelle 34
Misha 70


# Агрегация на коленке

посчитать средний пульс какого-нибудь участника со всех данны и сохранить его в новую переменную

In [83]:
def average_pr(person_id):
    st = r.hget('person:'+str(person_id), 'start_time').decode()
    mt = r.get('cur_time').decode()
    
    sum_pulse = 0
    
    for i in range(int(st), int(mt) + 1):
        pulse = r.hget(str(person_id)+':'+str(i), 'pulse_rate').decode()
        sum_pulse += int(pulse)
    
    avg_pr = sum_pulse/(int(mt) + 1 - int(st))
    print(avg_pr)
    
    r.set('avg_pulse:'+str(person_id), avg_pr)
    

In [84]:
for i in r.lrange('person_ids', 0, -1):
    average_pr(i.decode())

150.4111111111111
153.14444444444445
158.07777777777778
145.6888888888889
159.9
166.86
151.43
149.39
141.91
136.63
158.8
134.61


# DELETE и PIPELINE

удалить всю инфу об одном человеке

In [85]:
def cancel(person_id): #удалить среднее, все заметки, личную инфу, из списка
    
    pipe=r.pipeline()
    pipe.delete('avg_pulse:'+str(person_id))
    pipe.delete('person:'+str(person_id))
    
    st = r.hget('person:'+str(person_id), 'start_time')
    st = st.decode()
    mt = r.get('cur_time')
    mt = mt.decode()
        
    for i in range(int(st), int(mt) + 1):
        pipe.delete(str(person_id)+':'+str(i))
        
    pipe.lrem('person_ids', 0, person_id)
    
    pipe.execute()
        
    
    


In [86]:
cancel(3)

In [87]:
r.exists('person:3')

0

In [88]:
r.lrange('person_ids', 0 , -1)

[b'62444888524f204a15ee4a3a',
 b'62444888524f204a15ee4a39',
 b'62444888524f204a15ee4a38',
 b'62444888524f204a15ee4a37',
 b'62444888524f204a15ee4a36',
 b'6',
 b'5',
 b'4',
 b'2',
 b'1',
 b'0']

In [89]:
show_redis_data(r)

HTML(value='<table border="1"><tr><td style="padding:3px;width:150px;">4:23</td><td style="padding:3px;width:5…